http://preview.d2l.ai/d2l-en/master/chapter_generative-adversarial-networks/gan.html

In [1]:
%matplotlib inline
from d2l import tensorflow as d2l
import tensorflow as tf

https://www.tensorflow.org/api_docs/python/tf/random/normal

https://www.tensorflow.org/api_docs/python/tf/norm

In [2]:
X = d2l.normal([1000, 2], 0.0, 1, tf.float32)

In [3]:
A = d2l.tensor([[1, 2], [-0.1, 0.5]], tf.float32)
b = d2l.tensor([1, 2], tf.float32)
data = d2l.matmul(X, A) + b